In [85]:
# librerías
import pandas as pd
import re
import os

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize

from datasets import Dataset, DatasetDict

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oscarperez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/oscarperez/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/Users/oscarperez/opt/anaconda3/envs/acobur/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [86]:
import pandas as pd

ruta_archivo = 'oraciones.txt'
with open(ruta_archivo, 'r', encoding='utf-8') as archivo:
    contenido = archivo.read()

# Separar el contenido en filas y luego dividir cada fila por el delimitador '\t'
filas = [fila.split('\t') for fila in contenido.split('\n') if fila]

# Crear un DataFrame a partir de la lista de listas
# pandas automáticamente asignará cada elemento de las sublistas a una columna separada
df = pd.DataFrame(filas)

# Si sabes que siempre habrá 2 columnas, puedes renombrarlas para mayor claridad
if len(df.columns) == 2:
    df.columns = ['text', 'NombreArchivo']

print(df)


                                                    text  \
0                                REPÚBLICA DE COLOMBIA     
1                                                          
2                                                          
3                                                          
4                                                          
...                                                  ...   
47839                               DIANA FAJARDO RIVERA   
47840                                         Magistrada   
47841  Este documento fue generado con firma electrón...   
47842  Código de verificación: aca4c2c8090b666ff40ada...   
47843  Verifique este documento electrónico en: https...   

                                         NombreArchivo  
0                                                 None  
1                                                 None  
2                                                 None  
3                                                 N

In [104]:
df['conteo'] = range(len(df))
df = df.sort_values(by = 'conteo', ascending = False)

# Reemplazar 'None' por espacio vacío en la columna 'NombreArchivo'
df['NombreArchivo'] = df['NombreArchivo'].fillna('')

# Completar las filas vacías en 'NombreArchivo' con el último valor no vacío
df['NombreArchivo'] = df['NombreArchivo'].replace('', pd.NA).fillna(method='ffill')
#df = df.sort_values(by = 'conteo', ascending = True)

df.to_csv('sentencias.csv', index = False)

df.head(10)

/var/folders/dt/wd8bxjhn0_121hnh8db2qth40000gn/T/ipykernel_23525/157975696.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['NombreArchivo'] = df['NombreArchivo'].replace('', pd.NA).fillna(method='ffill')


,text,NombreArchivo,conteo
47843,Verifique este documento electrónico en: https...,D0015639-Auto Rechazo-(2024-01-25 07-04-41).pdf,47843
47842,Código de verificación: aca4c2c8090b666ff40ada...,D0015639-Auto Rechazo-(2024-01-25 07-04-41).pdf,47842
47841,Este documento fue generado con firma electrón...,D0015639-Auto Rechazo-(2024-01-25 07-04-41).pdf,47841
47840,Magistrada,D0015639-Auto Rechazo-(2024-01-25 07-04-41).pdf,47840
47839,DIANA FAJARDO RIVERA,D0015639-Auto Rechazo-(2024-01-25 07-04-41).pdf,47839
47838,Notifíquese y cú mplase.,D0015639-Auto Rechazo-(2024-01-25 07-04-41).pdf,47838
47837,"Ejecutoriada esta decisión, archívese el exped...",D0015639-Auto Rechazo-(2024-01-25 07-04-41).pdf,47837
47836,TERCERO.,D0015639-Auto Rechazo-(2024-01-25 07-04-41).pdf,47836
47835,lo previsto por el artículo 6º del Decreto 206...,D0015639-Auto Rechazo-(2024-01-25 07-04-41).pdf,47835
47834,procede el recurso de súplica ante la Sala Ple...,D0015639-Auto Rechazo-(2024-01-25 07-04-41).pdf,47834


In [101]:
# Convertir el DataFrame de pandas a un Dataset de Hugging Face
texto = df['text'] 
texto = texto.to_frame()
dataset = Dataset.from_pandas(texto)
dataset = dataset.remove_columns('__index_level_0__')


# Crear un DatasetDict con el Dataset bajo la clave 'train'
dataset_dict = DatasetDict({
    'train': dataset
})

print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 47844
    })
})


In [99]:
def clean_text(sentence_batch):
    # extrae el texto de la entrada
    text_list = sentence_batch['text']

    cleaned_text_list = []
    for text in text_list:
        # Convierte el texto a minúsculas
        text = text.lower()

        # Elimina URLs
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

        # Elimina las menciones @ y '#' de las redes sociales
        text = re.sub(r'\@\w+|\#\w+', '', text)

        # Elimina los caracteres de puntuación
        text = strip_punctuation(text)

        # Elimina los números
        text = strip_numeric(text)

        # Elimina las palabras cortas
        text = strip_short(text,minsize=2)

        # Elimina las palabras comunes (stop words)
        stop_words = set(stopwords.words('spanish'))
        word_tokens = word_tokenize(text)
        filtered_text = [word for word in word_tokens if word not in stop_words]

        cleaned_text_list.append(filtered_text)

    # Devuelve el texto limpio
    return {'text': cleaned_text_list}

In [102]:
sentences_corpus = dataset_dict.map(clean_text, batched = True)

Map:   0%|                                     | 0/47844 [00:00<?, ? examples/s]


NameError: name 'strip_punctuation' is not defined